In [2]:
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
def getDraftees():
    # get initial page
    url = "https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1989&college_id=0&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&order_by=year_id"
    r = requests.get(url).text
    soup = BeautifulSoup(r, 'lxml')
    
    # scrape headers
    headers = soup.find(id="stats").find('thead').find_all('tr')[1:]
    colNames = []
    for h in headers:
        cols = h.find_all('th')
        for c in cols:
            colNames.append(c.get_text())
    colNames = colNames[1:]
    # scrape individual rows
    rows = soup.find(id="stats").find('tbody').find_all('tr')
    stats = []
    for r in rows:
        statsoup = r.find_all('td')
        statRow = []
        for stat in statsoup:
            statRow.append(stat.get_text())
        stats.append(statRow)
    
    # scrape the remaining pages getting only data rows
    for offset in range(100, 1700, 100):
        offsetURL = "https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1989&year_max=&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=year_id&order_by_asc=&offset=%d" % (offset)
        offsetR = requests.get(offsetURL).text
        soup = BeautifulSoup(offsetR, 'lxml')
        rows = soup.find(id="stats").find('tbody').find_all('tr')
        for r in rows:
            statsoup = r.find_all('td')
            statRow = []
            for stat in statsoup:
                statRow.append(stat.get_text())
            stats.append(statRow)
        
    draftDF = pd.DataFrame(stats, columns = colNames)
    draftDF = draftDF.apply(pd.to_numeric, errors = 'ignore')
    return draftDF

# draft = getDraftees()



In [4]:
print(draft.shape)
draftdf = draft[np.isfinite(draft['Year'])]
print (draftdf.shape)

(1818, 25)
(1682, 25)


In [6]:
draftdf[['Year','Rd','Pk']] = draftdf[['Year','Rd','Pk']].apply(pd.to_numeric,errors="coerce",downcast="integer")
draftdf.to_csv("nba-draft-1989-to-2017.csv")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [63]:
def getCBBStats(player):
    url = "http://www.sports-reference.com/cbb/players/%s-1.html" % (player)
    r = requests.get(url).text
    soup = BeautifulSoup(r, 'lxml')
    
    # get headers
    headers = soup.find(id='players_per_game').find('thead').find_all('th')
    colNames = [col.get_text() for col in headers]
    
    # get career stats
    statsoup = soup.find(id='players_per_game').find('tfoot').find_all('td')
    stats = []
    statRow = ['Career']
    for s in statsoup:
        statRow.append(s.get_text())
    stats.append(statRow)
    
    # get awards
    awards = soup.find_all('td', {'data-stat': 'awards'})
    allAwards = [a.get_text() for a in awards]
    print (awards)
    print(allAwards)

    df = pd.DataFrame(stats, columns = colNames)
    df = df.apply(pd.to_numeric, errors = 'ignore')
    return df

myDf = getCBBStats("doug-mcdermott")
myDf

[<td class="left " data-stat="awards"></td>, <td class="left " data-stat="awards">MVC POY</td>, <td class="left " data-stat="awards">MVC POY</td>, <td class="left " data-stat="awards">AP POY; NABC POY; TSN POY<span class="tooltip" tip="Other awards: USBWA POY; Big East POY; "><strong> ...</strong></span></td>, <td class="left " data-stat="awards"></td>]
['', 'MVC POY', 'MVC POY', 'AP POY; NABC POY; TSN POY ...', '']


,Season,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,AST,STL,BLK,TOV,PF,PTS,,SOS,,Awards
0,Career,Creighton,NaN,145,145,31.5,7.9,14.3,0.55,6.0,...,1.3,0.2,0.1,2.2,1.8,21.7,NaN,4.29,NaN,NaN
